In [1]:
# Buiding a NN model
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Doanloading the training ata from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.61MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 204kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.76MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.4MB/s]


In [3]:
# Doanload the test data from open datasets

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [4]:
# Define the batch size
batch_size = 64

# Create data loaders

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Creating models
# Accelerator usage
device = torch.accelerator.current_accelerator().type\
if torch.accelerator.is_available() else "cpu"

print("Device: {device}")

Device: {device}


In [6]:
# Define the model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward_pass(self,x):
    x = self.flatten(x)
    logits = self.linear_rely_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**Optimizing the model parameters**

In [7]:
loss_function = nn.CrossEntropyLoss()# Define the loss function which will drive the optimization
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)#the optimizer which takes the loss information and updates the weight
# Loss function -> Gradients -> Weights adjustments

In [8]:
def train(dataloader, model, loss_function, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    prediction = model(X)
    loss = loss_function(prediction, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)* len(X)
      print(f"loss:{loss:>7f} [{current:>5d}/{size:.>5d}]")


In [ ]:
def test(dataloader, model, loss_function):
  size = len(dataloader.dataset)
  num_of_batches = len(dataloader)
  model.eval()
